# Fibonacci Retracement in Python for trading

The Fibonacci Retracement is an analysis on the Fibonacci sequence and applies key ratios to determine where price corrections are likely to occurr. Usually this method helps traders decide when to take positions, it only helps with the entry point decisions, so it doesn't guess the trend nor the future price, but it's a help in trending markets. 

On a trending market, if we can identify two pivot points or fractal points and we already know the trend direction. The challenge is to know when to execute our position, when to enter the market. So for example, if we already know the trend is up and we're going long we want to buy, but we also need to minimize our risk and maximize our profit. This is where Fibonacci can be a helpful approach: we can identify the Fibonacci levels and wait for the price to retrace back to the golden zone, which is somewhere around 60% retracement distance.Sometimes 60% works fine and some other times the price might go back to as low as 70% retracement. As long as WE stick to a systematic approach, WE should be able to make profit and decrease your risk. 

The reason this method works is mainly the reward risk ratio, which is usually above 1.5, going in the direction of the trend.

Let's see this strategy in an example:


In [2]:
# Loading the data
!pip install pandas_ta

import pandas as pd
import pandas_ta as ta

# Reading the data
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")

# Evaluating NA values in df
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace= True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, lenght= 12)
df['EMA'] = ta.ema(df.close, lenght=150)
df.tail()

df=df[0:2000]

Defaulting to user installation because normal site-packages is not writeable


## Trend detection


In [4]:

EMAsignal = [0]*len(df) # Generating signals based on Exponential Moving Average (EMA) 
backcandles = 15 #the amount of candles can be changed depending of the timeframe

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i]>=df.EMA[i]): #This condition checks if the maximum of the open and close prices of the candle 
                                                    #is greater than or equal to the corresponding EMA value. 
                                                    #If this condition is true, it means that the price is above the EMA.
            dnt =0
        if min(df.open[i], df.close[i]<=df.EMA[i]): #This condition checks if the minimum of the open and close prices of the candle is 
                                                    #less than or equal to the corresponding EMA value. 
                                                     #If this condition is true, it means that the price is below the EMA.
            upt=0
    if upt== 1 and dnt==1: #If both 'upt' and 'dnt' are still equal to 1 after checking all the historical candles, 
                           #it means that there is no crossover signal detected,
                           # and the current row corresponds to a period of consolidation.
        EMAsignal[row]=3
    elif upt==1: #If 'upt' is equal to 1 but dnt is not, it means that there is an uptrend signal detected.
        EMAsignal[row]=2
    elif dnt==1: #If 'dnt' is equal to 1 but upt is not, it means that there is a downtrend signal detected
        EMAsignal[row]=1  

df['EMASignal'] = EMAsignal  

## Generating our Fibonacci signal

In [7]:
def generate_signal(df, l, backcandles, gap, zone_threshold, price_diff_threshold):
                     # where 'l' is the index of the current candle to test if it's a signal candle; 
                     # 'gap' is a limited number of candles that doesn't stretch back to the entire back candles number;
                     # 'zone threshold' represent the "noise" in the market
                     # 'price_diff_threshold' is used to detect this difference 
                     # between the levels of the different pivots or the different fractal points.
    
    max_price = df.high[l-backcandles:l-gap].max()
    min_price = df.low[l-backcandles:l-gap].min()
    index_max = df.high[l-backcandles:l-gap].idmax()
    index_min = df.low[l-backcandles:l-gap].idxmin()
    price_diff = max_price - min_price

    if (df.EMASignal[l] == 2
        and (index_min < index_max)
        and price_diff >price_diff_threshold):
        l1 = max_price - 0.62 * price_diff #position entry 0.62 (golden zone)
        l2 = max_price - 0.78 * price_diff # Stop Loss 0.78
        l3 = max_price - 0. * price_diff # profit
        if abs(df.close[1]-l1) < zone_threshold and df.high[l-gap:l].min()>l1:
            return (2, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)
        
    #for the down trend we are going to use the same approach
    elif (df.EMASignal[l] == 1
        and (index_min > index_max)
        and price_diff >price_diff_threshold):
        l1 = min_price - 0.62 * price_diff #position entry 0.62 (golden zone)
        l2 = min_price - 0.78 * price_diff # Stop Loss 0.78
        l3 = min_price - 0. * price_diff # profit
        if abs(df.close[l]-l1) < zone_threshold and df.low[l-gap:l].max()<l1:
            return (2, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)